# Double Pendulum

In this notebook, we will create a class that solves the double pendulum problem and see what the consequences are.

In order to do this, we first cosntruct the Lagrangian for the double pendulum. From (11.37) and (11.38) in the textbook, the Lagrangian for the double pendulum is:

$$\begin{align}
    \mathcal{L} &= T(\theta_1, \theta_2, \dot{\theta}_1, \dot{\theta}_2) - U(\theta_1, \theta_2) \\
                &= \frac{1}{2} (m_1 + m_2) L_1^2 \dot{\theta}_1^2
                    + m_2 L_1 L_2 \dot{\theta}_1 \dot{\theta}_2 \cos{(\theta_1-\theta_2)}
                    + \frac{1}{2} m_2 L_2^2 \dot{\theta}_2^2
                    - (m_1 + m_2) g L_1 (1-\cos{\theta_1})
                    - m_2 g L_2 (1-\cos{\theta_2}),
\end{align}$$

where $L_1$, $m_1$ and $L_2$, $m_2$ are the length and mass of each pendulum. 

From this, we will solve for $\ddot{\theta}_1$ and $\ddot{\theta}_2$ using the Euler-Lagrange equations

$$\begin{align}
    \frac{d}{dt} \frac{\partial \mathcal{L}}{\partial \dot{\theta}_1} &= \frac{\partial \mathcal{L}}{\partial \theta_1}, \text{ and} \\
    \frac{d}{dt} \frac{\partial \mathcal{L}}{\partial \dot{\theta}_2} &= \frac{\partial \mathcal{L}}{\partial \theta_2}.
\end{align}$$

Using our solutions, we can then construct the two vectors:

$$\begin{align}
    \vec{y} = \begin{bmatrix}\theta_1 \\ \dot{\theta}_1 \\ \theta_2 \\ \dot{\theta}_2\end{bmatrix} \text{ and }
    \frac{d\vec{y}}{dt}
        = \begin{bmatrix}\dot{\theta}_1 \\ \ddot{\theta}_1 \\ \dot{\theta}_2 \\ \ddot{\theta}_2\end{bmatrix}.
\end{align}$$

Both $\vec{y}$ and $d\vec{y}/dt$ will then be given to the scipy's solve_ivp function to find the angle at each given point.

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
from scipy.integrate import odeint, solve_ivp

import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

In [ ]:
class LagrangianDoublePendulum():
    """
    Pendulum class implements the parameters and Lagrange's equations for 
     a double pendulum.
     
    Parameters
    ----------
    L1 | float | length of the first simple pendulum
    m1 | float | mass of the first pendulum
    L2 | float | length of the second simple pendulum
    m2 | float | mass of the second pendulum
    g  | float | gravitational acceleration at the earth's surface

    Methods
    -------
    dy_dt(t, y)
        Returns the right side of the differential equation in vector y, 
        given time t and the corresponding value of y.
    """
    def __init__(self,
                 L1=1., m1=1.,
                 L2=1., m2=1.,
                 g=1.
                ):
        self.L1 = L1
        self.m1 = m1
        self.L2 = L2
        self.m2 = m2
        self.g = g
    
    def dy_dt(self, t, y):
        """
        This function returns the right-hand side of the diffeq: 
        [dphi/dt d^2phi/dt^2]
        
        Parameters
        ----------
        t | float       | time 
        y | float array | A 4-component vector with y[0] = theta_1(t),
                          y[1] = dtheta_1/dt, y[2] = theta_2, y[3]=dtheta_2/dt
            
        Returns
        -------
        dy/dt
        """
        L1 = self.L1
        L2 = self.L2
        m1 = self.m1
        m2 = self.m2
        g = self.g
        return [y[1],
                -((g*(2*m1+m2)*np.sin(y[0])+g*m2*np.sin(y[0]-2*y[2])
                   +2*m2*(L2*y[3]**2+L1*y[1]**2*np.cos(y[0]-y[2]))
                   *np.sin(y[0]-y[2]))
                  /(2*L1*(m1+m2-m2*np.cos(y[0]-y[2])**2))),
                y[3],
                (((m1+m2)*(L1*y[1]**2+g*np.cos(y[0]))+L2*m2*y[3]**2*np.cos(y[0]-y[2]))*np.sin(y[0]-y[2]))
                /(L2*(m1+m2-m2*np.cos(y[0]-y[2])**2))
                ]
    
    def solve_ode(self, t_pts,
                  theta_1_0, theta_1_dot_0,
                  theta_2_0, theta_2_dot_0,
                  abserr=1.0e-12, relerr=1.0e-12):
        """
        Solve the ODE given initial conditions.
        Specify smaller abserr and relerr to get more precision.
        """
        y = [theta_1_0, theta_1_dot_0, theta_2_0, theta_2_dot_0] 
        solution = solve_ivp(self.dy_dt, (t_pts[0], t_pts[-1]), 
                             y, t_eval=t_pts, 
                             atol=abserr, rtol=relerr)
        theta_1, theta_1_dot, theta_2, theta_2_dot = solution.y

        return theta_1, theta_1_dot, theta_2, theta_2_dot

In [ ]:
from plotting_functions import * # plotting functions from the Lagrangian_pendulum notebook

We will now create one of these pendulum objects and then solve for different sets of initial conditions. One of the sets of initial conditions will be for very small agnles, slightly offset from one-another. The other will be for large angles that are slightly offset from one-another. We should see that for the small angles, the motion over time is periodic and about the same, where for the large angles, the small difference in initial conditions will lead to wildly different chaotic motions.

In [ ]:
dps = LagrangianDoublePendulum ()

t_pts = np.arange (0.,200.,0.1)

sm_th1_1, sm_th1_dot_1, sm_th2_1, sm_th2_dot_1 = dps.solve_ode (t_pts, 0.01, 0., 0.1+0., 0.)
sm_th1_2, sm_th1_dot_2, sm_th2_2, sm_th2_dot_2 = dps.solve_ode (t_pts, 0.01, 0., 0.1+0.0001, 0.)

lrg_th1_1, lrg_th1_dot_1, lrg_th2_1, lrg_th2_dot_1 = dps.solve_ode (t_pts, np.pi/2., 0., np.pi/2., 0.)
lrg_th1_2, lrg_th1_dot_2, lrg_th2_2, lrg_th2_dot_2 = dps.solve_ode (t_pts, np.pi/2., 0., np.pi/2.+0.0001, 0.)

In [ ]:
# start the plot!
fig = plt.figure(figsize=(10,5))
    
# first plot: phi plot 
ax_a = fig.add_subplot(1,2,1)                  

start, stop = start_stop_indices(t_pts, 0., 200.)    
plot_y_vs_x(t_pts[start : stop], sm_th1_1[start : stop], 
            axis_labels=(r'$t$',r'$\theta_1(t)$'), 
            color='blue',
            label=None, 
            title=r'$\theta_1(t)$', 
            ax=ax_a)
plot_y_vs_x(t_pts[start : stop], sm_th1_2[start : stop], 
            axis_labels=(r'$t$',r'$\theta_1(t)$'), 
            color='orange',
            label=None, 
            title=r'$\theta_1(t)$', 
            ax=ax_a)
                              
# second plot: phi_dot plot 
ax_b = fig.add_subplot(1,2,2)                  

start, stop = start_stop_indices(t_pts, 0., 200.)    
plot_y_vs_x(t_pts[start : stop], sm_th2_1[start : stop], 
            axis_labels=(r'$t$',r'$\theta_2(t)$'), 
            color='blue',
            label=None, 
            title=r'$\theta_2(t)$', 
            ax=ax_b)
plot_y_vs_x(t_pts[start : stop], sm_th2_2[start : stop], 
            axis_labels=(r'$t$',r'$\theta_2(t)$'), 
            color='orange',
            label=None, 
            title=r'$\theta_2(t)$', 
            ax=ax_b)

fig.suptitle ('Small Angles')

fig.tight_layout()

# start the plot!
fig2 = plt.figure(figsize=(10,5))
    
# first plot: phi plot 
ax_a2 = fig2.add_subplot(1,2,1)                  

start, stop = start_stop_indices(t_pts, 0., 200.)    
plot_y_vs_x(t_pts[start : stop], lrg_th1_1[start : stop], 
            axis_labels=(r'$t$',r'$\theta_1(t)$'), 
            color='blue',
            label=None, 
            title=r'$\theta_1(t)$', 
            ax=ax_a2)
plot_y_vs_x(t_pts[start : stop], lrg_th1_2[start : stop], 
            axis_labels=(r'$t$',r'$\theta_1(t)$'), 
            color='orange',
            label=None, 
            title=r'$\theta_1(t)$', 
            ax=ax_a2)
                              
# second plot: phi_dot plot 
ax_b2 = fig2.add_subplot(1,2,2)                  

start, stop = start_stop_indices(t_pts, 0., 200.)    
plot_y_vs_x(t_pts[start : stop], lrg_th2_1[start : stop], 
            axis_labels=(r'$t$',r'$\theta_2(t)$'), 
            color='blue',
            label=None, 
            title=r'$\theta_2(t)$', 
            ax=ax_b2)
plot_y_vs_x(t_pts[start : stop], lrg_th2_2[start : stop], 
            axis_labels=(r'$t$',r'$\theta_2(t)$'), 
            color='orange',
            label=None, 
            title=r'$\theta_2(t)$', 
            ax=ax_b2)

fig2.suptitle ('Large Angles')

fig2.tight_layout()

Above we can clearly see that, when you don't start with small initial angles, the resulting motion quickly becomes chaotic and very dependent on initial conditions. We can also make animations of these two different scenarios to help with the visualization:

### Small Angle Animation

In [ ]:
%%capture

fig_anim = plt.figure()
ax_anim = fig_anim.add_subplot(1,1,1)
ax_anim.set_ylim ([-2.5,0.1])
ax_anim.set_xlim ([-2.5,2.5])
ax_anim.set_aspect (1)
ax_anim.axis ('off')

# By assigning the first return from plot to line_anim, we can later change
#  the values in the line.

x1 = np.sin(sm_th1_1)
y1 = -np.cos(sm_th1_1)
x2 = x1+np.sin(sm_th2_1)
y2 = y1-np.cos(sm_th2_1)

x1_2 = np.sin(sm_th1_2)
y1_2 = -np.cos(sm_th1_2)
x2_2 = x1_2+np.sin(sm_th2_2)
y2_2 = y1_2-np.cos(sm_th2_2)



line_anim1, = ax_anim.plot( [0,x1[0]],
                            [0,y1[0]],
                            color='#bb0000', marker='o', lw=0.5, zorder=50)
line_anim2, = ax_anim.plot( [ x1[0],  x2[0]],
                            [ y1[0],  y2[0]],
                            color='#bb0000', marker='o', lw=0.5)

line_anim1_2, = ax_anim.plot( [0,x1_2[0]],
                            [0,y1_2[0]],
                            color='#666666', marker='o', lw=0.5, zorder=50)
line_anim2_2, = ax_anim.plot( [ x1_2[0],  x2_2[0]],
                              [ y1_2[0],  y2_2[0]],
                            color='#666666', marker='o', lw=0.5)

fig_anim.tight_layout()

In [ ]:
def animate_pendulums(i):
    """This is the function called by FuncAnimation to create each frame,
        numbered by i.  So each i corresponds to a point in the t_pts
        array, with index i.
    """
    i = i
    line_anim1.set_data ([0, x1[i]],[0, y1[i]])
    line_anim2.set_data ([ x1[i], x2[i]],
                         [ y1[i], y2[i]])
    line_anim1_2.set_data ([0, x1_2[i]],[0, y1_2[i]])
    line_anim2_2.set_data ([ x1_2[i], x2_2[i]],
                           [ y1_2[i], y2_2[i]])
    return (line_anim1,)

In [ ]:
frame_interval = 20  # time between frames
frame_number = len (t_pts) # number of frames to include (index of t_pts)
anim = animation.FuncAnimation(fig_anim, 
                               animate_pendulums, 
                               init_func=None,
                               frames=frame_number, 
                               interval=frame_interval, 
                               blit=True,
                               repeat=False)

In [ ]:
HTML(anim.to_jshtml())

### Large Angle Animation

In [ ]:
%%capture

fig_anim = plt.figure(figsize=(5,5))
ax_anim = fig_anim.add_subplot(1,1,1)
ax_anim.set_ylim ([-2.5,2.5])
ax_anim.set_xlim ([-2.5,2.5])
ax_anim.set_aspect (1)
ax_anim.axis ('off')

# By assigning the first return from plot to line_anim, we can later change
#  the values in the line.

x1 = np.sin(lrg_th1_1)
y1 = -np.cos(lrg_th1_1)
x2 = x1+np.sin(lrg_th2_1)
y2 = y1-np.cos(lrg_th2_1)

x1_2 = np.sin(lrg_th1_2)
y1_2 = -np.cos(lrg_th1_2)
x2_2 = x1_2+np.sin(lrg_th2_2)
y2_2 = y1_2-np.cos(lrg_th2_2)



line_anim1, = ax_anim.plot( [0,x1[0]],
                            [0,y1[0]],
                            color='#bb0000', marker='o', lw=0.5, zorder=50)
line_anim2, = ax_anim.plot( [ x1[0],  x2[0]],
                            [ y1[0],  y2[0]],
                            color='#bb0000', marker='o', lw=0.5)

line_anim1_2, = ax_anim.plot( [0,x1_2[0]],
                            [0,y1_2[0]],
                            color='#666666', marker='o', lw=0.5, zorder=50)
line_anim2_2, = ax_anim.plot( [ x1_2[0],  x2_2[0]],
                              [ y1_2[0],  y2_2[0]],
                            color='#666666', marker='o', lw=0.5)

fig_anim.tight_layout()

In [ ]:
def animate_pendulums(i):
    """This is the function called by FuncAnimation to create each frame,
        numbered by i.  So each i corresponds to a point in the t_pts
        array, with index i.
    """
    i = i
    line_anim1.set_data ([0, x1[i]],[0, y1[i]])
    line_anim2.set_data ([ x1[i], x2[i]],
                         [ y1[i], y2[i]])
    line_anim1_2.set_data ([0, x1_2[i]],[0, y1_2[i]])
    line_anim2_2.set_data ([ x1_2[i], x2_2[i]],
                           [ y1_2[i], y2_2[i]])
    return (line_anim1,)

In [ ]:
frame_interval = 20  # time between frames
frame_number = len (t_pts) # number of frames to include (index of t_pts)
anim = animation.FuncAnimation(fig_anim, 
                               animate_pendulums, 
                               init_func=None,
                               frames=frame_number, 
                               interval=frame_interval, 
                               blit=True,
                               repeat=False)

In [ ]:
HTML(anim.to_jshtml())